In [1]:
import time
import copy
import numpy as np
import itertools
import math
from functools import reduce # python3 compatibility
from operator import mul
import json

In [2]:
# Functions for generating environment with capability
# generating task
def gen_tasks_RF(task_num, t_agents, max_reward = 100): # task reward DB, for each task, each coaltion value is a random number between (0,100)
    tasks = [] # tasks is a list of task reward dictinoary, key represents the agent allocation in binary sum, value is a random integer
    for j in range(0, task_num):
        if j == task_num-1:
            coalitions = [(-1, 0)]
            tasks.append({key: value for (key, value) in coalitions})
        else:
            coalitions = list(itertools.chain(*[itertools.combinations(t_agents[j],i+1) for i,_ in enumerate(t_agents[j])]))
            dict_list = [(sum([2**a for a in com]), np.random.randint(1, max_reward+1)) for com in coalitions]
            tasks.append({key: value for (key, value) in dict_list})
    return tasks


def gen_tasks_RF2(task_num): # task reward DB, for each task, each coaltion value is a random number between (0,100)
    return [{} for j in range(0,task_num)]


def gen_constraints(agent_num, task_num, power=1, a_min_edge=2,
                    t_max_edge=5):  # power is the inforce you put in the probabilities

    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, the roughly each agent can work on two tasks)
    # calculate the max and min edges for agents
    seats = math.floor(t_max_edge * task_num)
    a_taskInds = [[] for i in range(0, agent_num)]
    t_counter = [0 for j in range(0, task_num)]  # each indicate the current number of agents on the task

    ## generating the number of tasks the agents could work on.
    a_taskNums = []
    for i in range(0, agent_num):
        a_max_edge = min((seats - (agent_num - 1 - i) * a_min_edge), t_max_edge)
        a_min_edge = min(a_min_edge, a_max_edge)
        a_taskNums.append(
            np.random.randint(a_min_edge, a_max_edge + 1))  # indicate the number of task the agent could work on
        seats -= a_taskNums[i]

    t_indexes = [j for j in range(0, task_num) if
                 t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.
    for i in range(0, agent_num):
        if any([tc == 0 for tc in t_counter]):
            t_prob = [(math.e ** (t_max_edge - t_counter[j])) ** power for j in
                      t_indexes]  # power is used to manify the probability
            sum_prob = sum(t_prob)
            t_prop_2 = [prop / sum_prob for prop in t_prob]

            # draw tasks accounting to their current allocations
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False,
                                                  p=[prop / sum_prob for prop in t_prob]))
            # increase the chosen task counters
        else:
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False))

        for j in a_taskInds[i]:
            t_counter[j] += 1
        t_indexes = [j for j in range(0, task_num) if
                     t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.

    # get also the list of agents for each task
    t_agents = [[i for i in range(0, agent_num) if j in a_taskInds[i]] for j in range(0, task_num)]

    return a_taskInds, t_agents


In [3]:
def task_reward_RF(task, coalition, gamma=1, max_reward=100): # task is a dict (reward DB), coalition is a list of agents (indexes)
    key = sum([2**int(coalition[i]) for i in range(0,len(coalition))])
    if key in task.keys():
        return task[key]
    else:
        task[key] = np.random.randint(0,max_reward+1)
        return task[key]
    

def sys_rewards_tasks(tasks, CS, gamma = 1): # CS is a list of coalitions
    return sum([task_reward_RF(tasks[j],CS[j],gamma) for j in range(0,len(tasks))])



def alloc_to_CS(tasks,alloc):
    task_num = len(tasks)
    CS = [[] for j in range(0, len(tasks))]
    for i in range(0,len(alloc)):
        if alloc[i]< task_num: # means allocated (!=task_num)
            CS[alloc[i]].append(i)
    return CS


def resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma=1):
    a_taskInds = constraints[0]
    task_num = len(tasks)

    a_msg_sum = [{d_key: sum([r_msgs[j][i][0] for j in a_taskInds[i] if j != d_key])
                         + r_msgs[d_key][i][1] for d_key in a_taskInds[i]}
                 for i in range(0, agent_num)]

    alloc = [max(ams, key=ams.get) if ams != {} else task_num for ams in a_msg_sum]
    return alloc, sys_rewards_tasks(tasks, alloc_to_CS(tasks, alloc), gamma), iteration, iter_over, converge


# Agent contribution
def agent_con_RF(task, j, a_tasks, query_agentIndex, cur_coalition, gamma=1): # cur_coalition is a list of agents (indexes)
    if j not in a_tasks[query_agentIndex]:
        return -1000
    else:
        if query_agentIndex in cur_coalition:
            new_coalition = list(cur_coalition)
            new_coalition.remove(query_agentIndex)
            return task_reward_RF(task,cur_coalition,gamma) - task_reward_RF(task,new_coalition,gamma)
        else:
            return task_reward_RF(task,cur_coalition+[query_agentIndex],gamma) - task_reward_RF(task,cur_coalition,gamma)


def aim_tasks(movement_values, a_tasks, a_index):
    max_mv = max(movement_values['agent ' + str(a_index)])
    target_tasks = []  # aiming tasks are the tasks the agent makes proposal to
    max_num = movement_values['agent ' + str(a_index)].count(max_mv)
    first_pos = 0
    for ind in range(max_num):
        new_list = movement_values['agent ' + str(a_index)][first_pos:]
        index_add_t = first_pos + new_list.index(max_mv)
        act_task = a_tasks[a_index][index_add_t]
        target_tasks.append(act_task)
        next_pos = new_list.index(max_mv) + 1
        first_pos += next_pos
    return target_tasks


def accepted_tasks(instruction_received, propose_states, a_index):
    accept_num = instruction_received['agent ' + str(a_index)].count(1)
    accept_tasks = []  # the index of accept message, which corresponding to the index of sending tasks
    first_pos = 0
    for ind in range(accept_num):
        new_list = instruction_received['agent ' + str(a_index)][first_pos:]
        new_t_ind = first_pos + new_list.index(1)
        act_task = propose_states['agent ' + str(a_index)][new_t_ind]
        accept_tasks.append(act_task)
        next_pos = new_list.index(1) + 1
        first_pos += next_pos
    return accept_tasks



In [4]:
def OPD_RF(agent_num, tasks, constraints, gamma):
    task_num = len(tasks)
    a_taskInds = [list(con) for con in constraints[0]]
    t_agentInds = [list(con) for con in constraints[1]]

    a_ubs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]
    a_lbs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]

    for j in range(0, task_num):

        linked_agentInds = t_agentInds[j]
        com_dict = []
        com_rewards = []
        for c in itertools.product(*[[0, 1] for i in linked_agentInds]):
            com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
            com_rewards.append(
                task_reward_RF(tasks[j], [a_key for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1], gamma)
            )

        for i in t_agentInds[j]:
            t_ind = a_taskInds[i].index(j)
            cons_j = [com_rewards[c]
                      for c in range(0, len(com_dict)) if com_dict[c][i] == 1]

            a_lbs[i][t_ind] = min(cons_j)
            a_ubs[i][t_ind] = max(cons_j)

    for i in range(0, agent_num):
        t_flag = [True for j in a_taskInds[i]]
        for t_ind in range(0, len(a_taskInds[i])):
            for t2_ind in range(0, len(a_taskInds[i])):
                if t_ind != t2_ind and a_ubs[i][t_ind] < a_lbs[i][t2_ind]:
                    t_flag[t_ind] = False
                    break

        for t_ind in range(0, len(t_flag)):
            if not t_flag[t_ind]:
                t_agentInds[a_taskInds[i][t_ind]].remove(i)

        new_a_taskInds = [a_taskInds[i][t_ind]
                          for t_ind in range(0, len(a_taskInds[i])) if t_flag[t_ind]]
        a_taskInds[i] = new_a_taskInds
    return a_taskInds, t_agentInds


def FMS_RF(agent_num, tasks, constraints, gamma, time_bound):
    converge = False
    iter_over = False
    start_time = time.time()
    a_taskInds = constraints[0]
    t_agentInds = constraints[1]
    task_num = len(tasks)

    q_msgs = [{t_key: {} for t_key in a_taskInds[i]} for i in range(0, agent_num)]
    r_msgs = [
        {t_agentInds[j][i]: ({1: -100} if len(a_taskInds[t_agentInds[j][i]]) == 1 else {key: -100 for key in [0, 1]})
         for i in range(0, len(t_agentInds[j]))}
        for j in range(0, task_num)]

    q_flags = [False for i in range(0, agent_num)]
    r_flags = [False for j in range(0, task_num)]

    iteration = 0
    while True:
        if time.time() - start_time >= time_bound:
            return resultCal_RF(agent_num, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)
        iteration += 1

        if iteration > agent_num + task_num:
            iter_over = True
            return resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)

        if all(q_flags) and all(r_flags):  # converge, msgs are all the same.
            converge = True
            break
        for i in range(0, agent_num):
            linked_taskInds = a_taskInds[i]

            flag = True
            for t_key in linked_taskInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge,
                                     gamma)
                ####### check time bound
                msgs = {}

                if len(linked_taskInds) > 1:
                    msgs[1] = sum([m[0] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
                    msg_0 = []
                    ts = list(linked_taskInds)
                    ts.remove(t_key)

                    for k in ts:
                        msg_0.append(sum([m[0] for m in [r_msgs[j][i] for j in ts if j != k]])
                                     + r_msgs[k][i][1])

                    msgs[0] = (0 if msg_0 == [] else max(msg_0))
                else:
                    msgs[1] = 0

                alphas = -sum(msgs.values()) / len(msgs.keys())

                msgs_regularised = {d_key: msgs[d_key] + alphas for d_key in msgs.keys()}

                old_msg = q_msgs[i][t_key]
                if old_msg != {} and any([abs(msgs_regularised[d_key] - old_msg[d_key]) > 10 ** (-5)
                                          for d_key in old_msg.keys()]):
                    flag = False

                q_msgs[i][t_key] = msgs_regularised

            if flag:  # agent i sending the same info
                q_flags[i] = True

        if time.time() - start_time >= time_bound:
            break
        ###################### SAME thing, using comprehension
        #             msgs = {t_key:{d_key:sum([m[d_key] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        #             alphas = {t_key:-sum(msgs[t_key].values())/len(msgs.keys())
        #                       for t_key in linked_taskInds}
        #             msgs_regularised = {t_key:{d_key:msgs[t_key][d_key] + alphas[t_key]
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        for j in range(0, task_num):
            linked_agentInds = t_agentInds[j]
            msg_con = [q_msgs[a][j] for a in linked_agentInds]

            com_dict = []
            com_rewards = []
            dom_com = [[0, 1] if len(a_taskInds[i]) > 1 else [1] for i in linked_agentInds]

            for c in itertools.product(*dom_com):
                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge,
                                     gamma)
                ####### check time bound

                com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
                com_rewards.append(
                    task_reward_RF(tasks[j], [a_key for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1], gamma)
                )

            flag = True
            for a_key in linked_agentInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge,
                                     gamma)
                ####### check time bound

                old_msg = r_msgs[j][a_key]
                q_table = []
                for c in range(0, len(com_dict)):
                    q_table.append(sum([q_msgs[a][j][com_dict[c][a]] for a in linked_agentInds if a != a_key])
                                   + com_rewards[c])

                r_msgs[j][a_key] = {
                    d_key: max([q_table[c] for c in range(0, len(com_dict)) if com_dict[c][a_key] == d_key])
                    for d_key in ([0, 1] if len(a_taskInds[a_key]) > 1 else [1])}

                if any([abs(r_msgs[j][a_key][d_key] - old_msg[d_key]) > 10 ** (-5) for d_key in old_msg.keys()]):
                    flag = False

            if flag:  # task j sending the same info
                r_flags[j] = True

    return resultCal_RF(agent_num, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)

In [5]:
def DSA_RF(constraints, tasks, agent_num, probability, time_bound):
    converge = False
    task_num = len(tasks)
    a_tasks = copy.deepcopy(constraints[0])
    [a_tasks[member].append(task_num) for member in range(0, agent_num)]
    t_agents = constraints[1]
    received_contribution = {} # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i]), int))
    movement_values = {}
    CS = [[] for j in range(0, task_num+1)]
    state = [task_num for i in range(0, agent_num)]
    update_t = range(0, task_num)
    iter = 0
    message_pass_num = 0
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        if time.time() - start_time >= time_bound:
            break
        iter += 1

        # update agents' allocation state
        for j in range(0, task_num):
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                ini_con_list = []
                for i in t_agents[j]:
                    message_pass_num += 1
                    current_coal = CS[j]
                    if len(current_coal) == 0:
                        ini_contrib = task_reward_RF(tasks[j], [i], max_reward=100)
                    else:
                        coal = [mem for mem in current_coal]
                        cur_reward = task_reward_RF(tasks[j], coal, gamma=1)
                        if i in current_coal:
                            remained_mems = current_coal[:]
                            remained_mems.remove(i)
                            if len(remained_mems) == 0:
                                non_a_reward = 0
                            else:
                                remain_coal = [remai_mem for remai_mem in remained_mems]
                                non_a_reward = task_reward_RF(tasks[j], remain_coal, gamma=1)
                            ini_contrib = cur_reward - non_a_reward
                        else:
                            coal.append(i)
                            add_reward = task_reward_RF(tasks[j], coal, gamma=1)
                            ini_contrib = add_reward - cur_reward
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = ini_contrib
                    i_ind = t_agents[j].index(i)

        # agents calculate their movement values #
        for i in range(0, agent_num):
            current_task = state[i]   # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind]-current_contribution for mess_ind in range(0, len(received_contribution['agent ' + str(i)]))]

        # agents choose their tasks  ##########
        update_t = []
        max_mv = [[] for i in range(0, agent_num)]
        for i in range(0, agent_num):
            max_mv[i] = max(movement_values['agent ' + str(i)])
            if max_mv[i] > 0:
                pro = np.random.random(1)
                if pro > 1 - probability:
                    old_task = state[i]
                    potential_task = []
                    max_num = movement_values['agent ' + str(i)].count(max_mv[i])
                    first_pos = 0
                    for ind in range(max_num):
                        new_list = movement_values['agent ' + str(i)][first_pos:]
                        poten_ind = first_pos + new_list.index(max_mv[i])
                        potential_task.append(a_tasks[i][poten_ind])
                        next_pos = new_list.index(max_mv[i]) + 1
                        first_pos += next_pos
                    choose_t_ind = np.random.randint(len(potential_task))
                    choosed_task = potential_task[choose_t_ind]
                    CS[choosed_task].append(i)
                    update_t.append(choosed_task)
                    state[i] = choosed_task
                    if old_task != task_num:
                        CS[old_task].remove(i)
                        update_t.append(old_task)
        #  if continue
        if max(max_mv) > 0:
            is_continue = True
        else:
            is_continue = False
            converge = True

    #  current system reward    #
    ini_task_u = [[] for i in range(0, task_num)]
    for j in range(0, task_num):
        if len(CS[j]) == 0:
            ini_task_u[j] = 0
        else:
            t_coalition = [c_mem for c_mem in CS[j]]
            ini_task_u[j] = task_reward_RF(tasks[j], t_coalition, gamma=1)
    global_u = sum(ini_task_u)

    return global_u, iter, message_pass_num, converge


In [6]:
def disNE_RF(constraints, tasks, agent_num, time_bound, CS=[]):
    converge = False
    task_num = len(tasks)
    a_tasks = copy.deepcopy(constraints[0])
    t_agents = copy.deepcopy(constraints[1])
    t_agents.append(list(range(0, agent_num)))
    
    if CS==[]:
        CS = [[] for j in range(0, task_num+1)]
        CS[task_num] = list(range(0, agent_num))

    received_contribution = {} # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i]), int))
    movement_values = {}
    
    state = [task_num for i in range(0, agent_num)]
    update_t = range(0, task_num)
    iter = 0
    message_pass_num = 0
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        if time.time() - start_time >= time_bound:
            break
        iter += 1

        # update agents' allocation state
        for j in range(0, task_num):
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                ini_con_list = []
                for i in t_agents[j]:
                    message_pass_num += 1
                    current_coal = CS[j]
                    if len(current_coal) == 0:
                        ini_contrib = task_reward_RF(tasks[j], [i], max_reward=100)
                    else:
                        coal = [mem for mem in current_coal]
                        cur_reward = task_reward_RF(tasks[j], coal, gamma=1)
                        if i in current_coal:
                            remained_mems = current_coal[:]
                            remained_mems.remove(i)
                            if len(remained_mems) == 0:
                                non_a_reward = 0
                            else:
                                remain_coal = [remai_mem for remai_mem in remained_mems]
                                non_a_reward = task_reward_RF(tasks[j], remain_coal, gamma=1)
                            ini_contrib = cur_reward - non_a_reward
                        else:
                            coal.append(i)
                            add_reward = task_reward_RF(tasks[j], coal, gamma=1)
                            ini_contrib = add_reward - cur_reward
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = ini_contrib
                    i_ind = t_agents[j].index(i)

        # agents calculate their movement values #
        for i in range(0, agent_num):
            # print('i: ', i)
            current_task = state[i]  # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind] - current_contribution for mess_ind in
                range(0, len(received_contribution['agent ' + str(i)]))]


        max_mv = [max(movement_values['agent ' + str(i)]) for i in range(0, agent_num)]
        # check whether or not to continue  #
        if max(max_mv) <= 0:  # in this case, no agent will propose
            is_continue = False
            converge = True
        if is_continue:
            # agents propose to tasks #
            received_proposal = [{'from agents': [], 'proposal values': []} for j in range(0, task_num)]  # received proposals
            propose_states = {}  # which agents propose to which tasks
            propose_agents = []  # which agents make proposals
            received_tasks = []  # which tasks receive proposals
            for i in range(0, agent_num):
                if max_mv[i] > 0:
                    propose_agents.append(i)
                    pick_tasks = [k for k, x in enumerate(movement_values['agent ' + str(i)]) if x == max_mv[i]]
                    aim_tasks = [a_tasks[i][k] for k in pick_tasks]
                    old_task = state[i]
                    if old_task != task_num:
                        aim_tasks.append(old_task)
                    propose_states['agent ' + str(i)] = aim_tasks
                    received_tasks.append(aim_tasks)
                    for j in aim_tasks:
                        received_proposal[j]['from agents'].append(i)
                        received_proposal[j]['proposal values'].append(max_mv[i])
                    message_pass_num += len(aim_tasks)


            #  find out which coalitions received proposal
            active_tasks = []
            for j in received_tasks:
                active_tasks.extend(j)
            active_tasks = list(set(active_tasks))
            # tasks send instruction messages to agents #
            instruction_received = {}  # instruction received by the proposed agents
            for i in propose_agents:
                instruction_received['agent ' + str(i)] = [0] * len(propose_states['agent ' + str(i)])  # 0: reject, 1: accept

            for j in active_tasks:
                message_pass_num += len(received_proposal[j]['from agents'])
                max_pro = max(received_proposal[j]['proposal values'])
                candi_agents = [k for k, x in enumerate(received_proposal[j]['proposal values']) if x == max_pro]
                poent_accep_agents = [received_proposal[j]['from agents'][k] for k in
                                      candi_agents]  # the agents who give the maximum proposal
                choosed_agent = np.random.choice(poent_accep_agents)  # the agent index accepted by task j
                acc_t_ind = propose_states['agent ' + str(int(choosed_agent))].index(j)  # the index of task j
                instruction_received['agent ' + str(int(choosed_agent))][acc_t_ind] = 1

            # agents move to tasks
            update_t = []
            for i in propose_agents:
                accept_num = instruction_received['agent ' + str(i)].count(1)
                if accept_num == 0:
                    continue
                else:
                    accept_ind = [k for k, x in enumerate(instruction_received['agent ' + str(i)]) if x == 1]
                    accept_tasks = [propose_states['agent ' + str(i)][k] for k in accept_ind]
                    ########### moving condition ############
                    # i) the agent has not been allocated and it receives 1 (accept) from its target-task(s).
                    # ii) the agent has a current task and it receives 1 from both its target-task(s) and its current task.
                    old_coa = state[i]
                    if old_coa != task_num:
                        old_coa_ind = propose_states['agent ' + str(i)].index(old_coa)
                        if instruction_received['agent ' + str(i)][
                            old_coa_ind] == 0:  # the agent's own task reject its movement
                            continue
                        else:
                            accept_tasks.remove(old_coa)  # the other accept-coalitions besides its own coalition
                    if len(accept_tasks) == 0:
                        continue
                    else:
                        choosed_task = np.random.choice(accept_tasks)  # random select a coalition that gives accept message
                        CS[choosed_task].append(i)
                        CS[old_coa].remove(i)
                        message_pass_num += 1
                        update_t.append(choosed_task)
                        if old_coa != task_num:
                            message_pass_num += 1
                            update_t.append(old_coa)

    # current system reward #
    ini_task_u = [[] for j in range(0, task_num)]
    for j in range(0, task_num):
        if len(CS[j]) == 0:
            ini_task_u[j] = 0
        else:
            t_coalition = [c_mem for c_mem in CS[j]]
            ini_task_u[j] = task_reward_RF(tasks[j], t_coalition, gamma=1)
    global_u = sum(ini_task_u)

    return global_u, iter, message_pass_num, converge


In [7]:
def random(agent_num,tasks,constraints, gamma = 1):
    task_num = len(tasks)
    a_taskInds = copy.deepcopy(constraints[0])
    alloc = [np.random.choice(a_taskInds[i]+[task_num]) for i in range(0,agent_num)]
    return alloc, sys_rewards_tasks(tasks, alloc_to_CS(tasks,alloc), gamma)

In [8]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f)
        f.write('\n')
        # f.write(os.linesep)
        f.close()

In [ ]:
run_num = 100
probability = 0.7
time_bound = 300
max_t_num = 1000
capNum = 10
max_capNum_task = 10
max_capNum_agent = 10

a_min_edge = 1
min_t_num = 100
ex_identifier = 0


# compare algorithms
for run in range(0, run_num):
    print('run:', run)
    task_num = min_t_num
    while task_num <= max_t_num:
        ex_identifier += 1
        t_max_edge = 0.04 * task_num
        agent_num = 2 * task_num
        constraints = gen_constraints(agent_num, task_num, 1, a_min_edge, t_max_edge) #max(M,N)
        t_agents = copy.deepcopy(constraints[1])
        t_agents.append(list(range(0, agent_num)))

        if t_max_edge <= 17:
            tasks = gen_tasks_RF(task_num, t_agents)
        else: 
            tasks = gen_tasks_RF2(task_num)


        result = {"ex_identifier": ex_identifier, "task_num": task_num, "agent_num": agent_num}

        
        ##  BnB-FMS
        if t_max_edge <= 17:
            start = time.time()
            new_con = OPD_RF(agent_num, tasks, constraints, gamma=1)
            r = FMS_RF(agent_num, tasks, new_con, gamma=1, time_bound=time_bound)
            end = time.time()
            result['BnBFMS_u'] = r[1]
            result['BnBFMS_t'] = end - start
            result['BnBFMS_iter'] = r[2]
            result['BnBFMS_converge'] = r[4]
            print("task_number: ", task_num, "BnBFMS time:", result['BnBFMS_t'],'result:',result['BnBFMS_u'],
                  "iteration:", result['BnBFMS_iter'], "converge?", result['BnBFMS_converge'])

        #  Rand
        r = random(agent_num, tasks,constraints, gamma=1)
        alloc = r[0]
        result['rand'] = r[1]
        print("task_number: ", task_num, 'rand result',result['rand'])
        rand_CS = [[] for i in list(range(0, task_num + 1))]
        for i in list(range(0, agent_num)):
            rand_CS[alloc[i]].append(i)


        ##  DSA
        start = time.time()
        dsa_u, dsa_iter, dsa_msg_num, if_converge = DSA_RF(constraints, tasks, agent_num, probability, time_bound)
        end = time.time()
        result['dsa_u'] = dsa_u
        result['dsa_t'] = end-start
        result['dsa_iter'] = dsa_iter
        result['dsa_msg'] = dsa_msg_num
        result['DSA_converge'] = if_converge
        print("task_number: ", task_num, "DSA time:", result['dsa_t'],'result:', result['dsa_u'],
              "iteration:", result['dsa_iter'], "messages", result['dsa_msg'], "converge?", result['DSA_converge'])
        print()
        
        
        ##  DisNE starts from a random solution
        rand_disne = copy.deepcopy(rand_CS) 
        start = time.time()
        LS_u, LS_iter, LS_msg_num, if_converge = disNE_RF(constraints, tasks, agent_num, time_bound, rand_disne)
        end = time.time()
        result['LS_u'] = LS_u
        result['LS_t'] = end - start
        result['LS_iter'] = LS_iter
        result['LS_msg_num'] = LS_msg_num
        result['LS_converge'] = if_converge
        print("task number: ", task_num, "LS_t:", result['LS_t'], 'LS_u:', LS_u,
                "LS_iter:", LS_iter, "messages", result['LS_msg_num'], "converge?", result['LS_converge'])
        print()
        
        ##  DisNE
        start = time.time()
        disNE_u, disNE_iter, disNE_msg_num, if_converge = disNE_RF(constraints, tasks, agent_num, time_bound, [])
        end = time.time()
        result['disNE_u'] = disNE_u
        result['disNE_t'] = end-start
        result['disNE_iter'] = disNE_iter
        result['DisNE_msg'] = disNE_msg_num
        result['DisNE_converge'] = if_converge
        print("task_number: ", task_num, "disNE time:", result['disNE_t'],'result:',result['disNE_u'],
              "iteration:", result['disNE_iter'], "messages", result['DisNE_msg'], "converge?", result['DisNE_converge'])
        print()

# #         # append data and result
#         files = {'gen_RF_thesis': [result, '']}
#         for filename in list(files.keys()):
#             append_record(files[filename][0], filename, typ=files[filename][1])

#         increase the task_number
        task_num += 100
    run += 1




run: 0
task_number:  100 BnBFMS time: 3.7016208171844482 result: 7278 iteration: 301 converge? False
task_number:  100 rand result 4715
task_number:  100 DSA time: 0.029215097427368164 result: 8079 iteration: 15 messages 1612 converge? True

task number:  100 LS_t: 0.01409769058227539 LS_u: 7878 LS_iter: 7 messages 1541 converge? True

task_number:  100 disNE time: 0.01145029067993164 result: 8009 iteration: 5 messages 1504 converge? True

